In [2]:
import os
import openai
import asyncio
import json
import tiktoken
import pandas as pd
from tqdm import tqdm


os.environ["OPENAI_API_KEY"] =''
openai.api_key=os.getenv("OPENAI_API_KEY")

data = pd.read_csv('/home/leedhn/nas2/PiCL_DPR/downloads/data/retriever/qas/nq-test-gold.csv', sep='\t', header=None)
data.columns=['question','passage']
all_data = pd.read_csv('/home/leedhn/nas2/PiCL_DPR/downloads/data/wikipedia_split/psgs_cf.tsv',sep='\t')
all_data.columns=['id','text','title']

ids = list(all_data['id'])
idx = [int(i[8:]) for i in ids]
all_data['idx']=idx
all_data.head()

,id,text,title,idx
0,gold_cf:1,"A sequel , Deadpool 3 , is in development .",Deadpool 2,1
1,gold_cf:2,No . overall No. in season Title Directed by W...,List of Scandal episodes,2
2,gold_cf:3,The Philadelphia Eagles are a professional Ame...,Philadelphia Eagles,3
3,gold_cf:4,"Dragon Ball Z ( ドラゴンボール ゼット , Doragon Bōru Zet...",List of Dragon Ball Z episodes,4
4,gold_cf:5,Wong George Huang FBI Special Agent 4 -- 12 2 ...,Law & Order: Special Victims Unit,5


In [3]:
data.head()

,question,passage
0,who got the first nobel prize in physics,['Wilhelm Conrad Röntgen']
1,when is the next deadpool movie being released,"['May 18 , 2018']"
2,when is the next scandal episode coming out,"['February 1 , 2018']"
3,when is the last time the philadelphia won the...,"['2017', 'Super Bowl LII ,']"
4,how many episodes are there in dragon ball z,"['291 episodes', '291']"


In [4]:
gold_data = pd.merge(data, all_data, left_index=True,right_on='idx', how='right').fillna('')
gold_data.head()

,question,passage,id,text,title,idx
0,when is the next deadpool movie being released,"['May 18 , 2018']",gold_cf:1,"A sequel , Deadpool 3 , is in development .",Deadpool 2,1
1,when is the next scandal episode coming out,"['February 1 , 2018']",gold_cf:2,No . overall No. in season Title Directed by W...,List of Scandal episodes,2
2,when is the last time the philadelphia won the...,"['2017', 'Super Bowl LII ,']",gold_cf:3,The Philadelphia Eagles are a professional Ame...,Philadelphia Eagles,3
3,how many episodes are there in dragon ball z,"['291 episodes', '291']",gold_cf:4,"Dragon Ball Z ( ドラゴンボール ゼット , Doragon Bōru Zet...",List of Dragon Ball Z episodes,4
4,cast of law & order special victim unit,"['Christopher Meloni', 'Mariska Hargitay', 'St...",gold_cf:5,Wong George Huang FBI Special Agent 4 -- 12 2 ...,Law & Order: Special Victims Unit,5


In [5]:

rest_idx = list(gold_data['idx'])

In [7]:
for index, line in (gold_data.loc[gold_data['idx'].isin(rest_idx)].iterrows()):
    print(line)
    break

question    when is the next deadpool movie being released
passage                                  ['May 18 , 2018']
id                                               gold_cf:1
text           A sequel , Deadpool 3 , is in development .
title                                           Deadpool 2
idx                                                      1
Name: 0, dtype: object


In [10]:
gold_data.loc[gold_data['idx']==212]å

,question,passage,id,text,title,idx
211,who wrote how do you do it by gerry and the pa...,['Mitch Murray'],gold_cf:212,`` How Do You Do It ? '' ( 1963 ) `` I Like It...,How Do You Do It?,212


In [ ]:



# async def call_gpt(cur_prompt,n):
#     ans = openai.ChatCompletion.create(
#     # ans = openai.Completion.create(
#                 # model="text-davinci-002",
#                 # model='text-davinci-003',
#                 model="gpt-3.5-turbo",
#                 max_tokens=512,
#                 # stop=['[STOP]'],
#                 messages=[{"role": "user", "content": cur_prompt}],
#                 # prompt=cur_prompt,
#                 top_p=0.95,
#                 n=n,
#                 temperature=0.5)

#     # print(ans)
#     # returned = [line['text'] for line in ans['choices']]#ans['choices'][0]['text']
#     returned = [line['message']['content'] for line in ans['choices']]
#     # print( greenify(returned), end='')
#     # print(ans)
    

#     return returned



# async def promptf(row, question, text, title, answer):
#   if title:
#     passage = text+' '+title
#   else:
#     passage=text
  
#   enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
#   passage_len = len(enc.encode(passage))
#   if passage_len > 2900: #truncate
#     passage = enc.decode(enc.encode(passage)[:2900])

#   # cur_prompt=f"Given question and passage,\n paraphrase the passage.\nQuestion: {question}\nPassage: {passage}\nParaphrasead passage: "
#   cur_prompt=f"Please paraphrase the passage. Not explaining.\nPassage: {passage}\nParaphrasead passage: "

#   # ret_text= await call_gpt(cur_prompt,1)
#   # cur_prompt = cur_prompt + ret_text[0]+'\n Paraphrased passage: '
#   print(f"{row} is generating...")
#   ret_texts = await call_gpt(cur_prompt,20)
  
#   with open(f"chatgpt_gen_cf/{row}.json",'w') as f:
#     json.dump(ret_texts, f)

#   return ret_texts

# # question="who got the first nobel prize in physics"
# # text="The first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad Röntgen , of Germany , who received 150,782 SEK , which is equal to 7,731,004 SEK in December 2007 . John Bardeen is the only laureate to win the prize twice -- in 1956 and 1972 . Maria Skłodowska - Curie also won two Nobel Prizes , for physics in 1903 and chemistry in 1911 . William Lawrence Bragg was , until October 2014 , the youngest ever Nobel laureate ; he won the prize in 1915 at the age of 25 . Two women have won the prize : Curie and Maria Goeppert - Mayer ( 1963 ) . As of 2017 , the prize has been awarded to 206 individuals . There have been six years in which the Nobel Prize in Physics was not awarded ( 1916 , 1931 , 1934 , 1940 -- 1942 ) ."
# # title="List of Nobel laureates in Physics"
# # promptf(0,question,text,title)



# from tqdm.asyncio import tqdm_asyncio

# async def run_chatgpt(indexes):
#     # for index, line in (gold_data.iterrows()):
#         # chatgpt_psg = []
#     chat_gen = [promptf(line['idx'],line['question'],line['text'],line['title'],line['answers'][0]) for index, line in (gold_data_with_ans.loc[gold_data_with_ans['idx'].isin(indexes)].iterrows())]
#         # gold_data[f'chatgpt_passage'][index]+=chat_gen
#     #, passage=f"{} {}"))
#     # print(gold_data[f'chatgpt_passage'][index])
#     # break
    
#     await tqdm_asyncio.gather(*chat_gen)

# async def main(indexes):
#       await run_chatgpt(indexes)



# if __name__=="__main__":
#   if len(rest_idx) > 200:
#     for start_idx in tqdm(range(0,len(rest_idx), 200)):
#       cur_idx = rest_idx[start_idx:start_idx+200]
#       asyncio.run(main(cur_idx))
#   else:
#     asyncio.run(main(rest_idx))




In [15]:
gold_psgs = pd.read_csv('/home/leedhn/nas2/PiCL_DPR/downloads/data/wikipedia_split/grels_gold_psg_title.csv',sep='\t')
gold_psgs.head()

,Unnamed: 0,question,pid,ptext,id,text,title
0,0,who got the first nobel prize in physics,gold:0,The first Nobel Prize in Physics was awarded i...,gold:0,The first Nobel Prize in Physics was awarded i...,List of Nobel laureates in Physics
1,1,when is the next deadpool movie being released,gold:1,Deadpool 2 is scheduled to be released in the ...,gold:1,Deadpool 2 is scheduled to be released in the ...,Deadpool 2
2,2,when is the next scandal episode coming out,gold:3,No . overall No. in season Title Directed by W...,gold:3,No . overall No. in season Title Directed by W...,List of Scandal episodes
3,3,when is the last time the philadelphia won the...,gold:4,The Philadelphia Eagles are a professional Ame...,gold:4,The Philadelphia Eagles are a professional Ame...,Philadelphia Eagles
4,4,how many episodes are there in dragon ball z,gold:7,"Dragon Ball Z ( ドラゴンボール ゼット , Doragon Bōru Zet...",gold:7,"Dragon Ball Z ( ドラゴンボール ゼット , Doragon Bōru Zet...",List of Dragon Ball Z episodes


In [20]:

PATH='/home/leedhn/nas2/PiCL_DPR/chatgpt_gen_cf_new'
file_list = os.listdir(PATH)
file_list = [item for item in file_list if 'json' in item]
file_list = [int(item[:-5]) for item in file_list]
file_list.sort()

for item in file_list:
    with open(f"{PATH}/{item}.json",'r') as f:
        data = json.load(f)
    real_idx = idx_maps[item]
    with open(f"{PATH}_cor/{real_idx}.json",'w') as f:
        json.dump(data,f)

KeyError: 1392

In [ ]:
import pandas as pd
import json
import os


PATH='/home/leedhn/nas2/PiCL_DPR/chatgpt_gen_cf_new'
file_list = os.listdir(PATH)
file_list = [item for item in file_list if 'json' in item]
file_list = [int(item[:-5]) for item in file_list]
file_list.sort()
gold_psgs = pd.read_csv('/home/leedhn/nas2/PiCL_DPR/downloads/data/wikipedia_split/grels_gold_psg_title.csv',sep='\t')

data=[[] for _ in range(20)]
for item in file_list:
    with open(f"{PATH}/{item}.json",'r') as f:
        chat_data = json.load(f)
    for i, chat in enumerate(chat_data):
        if f'gold:{item}' in list(gold_psgs['id']):
            data[i].append(
                {
                    'id':f'chat{i}:{item}',
                    'text': chat,
                    'title': list(gold_psgs.loc[gold_psgs['id']==f'gold:{item}']['title'])[0]
                }
            )
        else:
            data[i].append(
                {
                    'id':f'chat{i}:{item}',
                    'text': chat,
                    'title': ''
                }
            )

for i, d in enumerate(data):
    df = pd.DataFrame.from_dict(d)
    print(df.head())
    print(df.shape)
    df.to_csv(f'/home/leedhn/nas2/PiCL_DPR/downloads/data/wikipedia_split/chat_gen_cf_new_right_{i}.csv',sep='\t',index=False)
    